In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import shutil
from functions import extractEanCodShopfarma, GeraDataframe, incluirPreco_PP

In [15]:
def rotinaAlteracaoPrecos(nome_relatorio, cod_grupo=None, categoria=None, laboratorio=None, excluir_categoria=False, excluir_laboratorio=False):
    extractEanCodShopfarma(nome_relatorio, 'cadastro', cod_grupo, categoria, laboratorio, excluir_categoria, excluir_laboratorio)
    df_cadastro = GeraDataframe(f'{nome_relatorio}_dados_cadastro.xls', 'cadastro')
    extractEanCodShopfarma(nome_relatorio, 'valores', cod_grupo, categoria, laboratorio, excluir_categoria, excluir_laboratorio)
    df_valores = GeraDataframe(f'{nome_relatorio}_dados_valores.xls', 'valores')
    df = pd.merge(df_cadastro, df_valores, on='codshopfarma')
    df.drop(['produto_x'], axis=1, inplace=True)
    df.rename({'produto_y': 'produto'}, axis=1, inplace=True)
    df['margem'] = (df['preco_venda']/df['preco_compra']-1)*100
    df.to_excel(f'{nome_relatorio}.xls', index=False)
    print(df)
    #dfpp, lst_error, lst_no_pmc = incluirPreco_PP(df)
    #dfpp = pd.DataFrame(dfpp, columns=['ean', 'preco_pp'])
    #comparador = df.merge(dfpp, on='ean')
    #comparador['diferencapreco'] = comparador['preco_venda'] - comparador['preco_pp']
    #comparador['%diff'] = (comparador['diferencapreco'] / comparador['preco_venda']) * 100
    return df#comparador, lst_error, lst_no_pmc

def retornaPrecos(arquivo_xls, desconto=0):
    df = pd.read_excel(arquivo_xls)
    df['preco_retornado'] = (df['preco_venda'] - (df['preco_venda'] * (desconto))).astype(int) - 0.01
    return df

rel_retorno_anticoncepcionais = retornaPrecos('anticoncepcionais.xls', 0.15)
rel_retorno_anticoncepcionais

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get("http://179.187.94.169:4647/sgfpod1/Rel_0087.pod?cacheId=1676047264063")
sleep(1)

In [16]:
df = rotinaAlteracaoPrecos('genericos', cod_grupo=4000)

C:\Users\Samsung\AppData\Local\Temp\ipykernel_26780\3474854901.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(f'{nome_relatorio}.xls', index=False)


     codshopfarma        laboratorio           ean  \
0           16982  ACHE/BIOSINTETICA  7.896182e+12   
1          113267              CIMED  7.896523e+12   
2            8731        NEO QUIMICA  7.896714e+12   
3          110884              TEUTO  7.896112e+12   
4           16545       NOVA QUIMICA  7.895296e+12   
...           ...                ...           ...   
3076        16190             SANDOZ  7.897596e+12   
3077        16923                EMS  7.896005e+12   
3078        56518           NOVARTIS  7.897596e+12   
3079       134104             GERMED  7.896005e+12   
3080        89587          EUROFARMA  7.891317e+12   

                           produto  preco_venda  preco_compra      margem  
0        ACEBROFILINA XPE AD 120ML        36.50         12.57  190.373906  
1        ACEBROFILINA XPE AD 120ML        35.00         12.99  169.438029  
2        ACEBROFILINA XPE AD 120ML        32.50          7.99  306.758448  
3        ACEBROFILINA XPE AD 120ML        18.90

In [12]:
df

,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,margem
0,16982,ACHE/BIOSINTETICA,7.896182e+12,ACEBROFILINA XPE AD 120ML,36.50,12.57,290.373906
1,113267,CIMED,7.896523e+12,ACEBROFILINA XPE AD 120ML,35.00,12.99,269.438029
2,8731,NEO QUIMICA,7.896714e+12,ACEBROFILINA XPE AD 120ML,32.50,7.99,406.758448
3,110884,TEUTO,7.896112e+12,ACEBROFILINA XPE AD 120ML,18.90,10.02,188.622754
4,16545,NOVA QUIMICA,7.895296e+12,ACEBROFILINA XPE AD 120ML,37.05,0.00,inf
...,...,...,...,...,...,...,...
3076,16190,SANDOZ,7.897596e+12,ZOLPIDEM 10MG 30CP REV,82.00,8.48,966.981132
3077,16923,EMS,7.896005e+12,ZOLPIDEM 10MG 30CP REV,72.00,6.74,1068.249258
3078,56518,NOVARTIS,7.897596e+12,ZOLPIDEM 10MG 30CP REV,49.90,6.99,713.876967
3079,134104,GERMED,7.896005e+12,ZOLPIDEM 5MG 30CP SUBL,77.00,26.07,295.358650


C:\Users\Samsung\AppData\Local\Temp\ipykernel_26780\3275955702.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('genericos.xls', index=False)


In [ ]:
df['pp+20%'] = df['preco_pp'] * 1.2
df["compra+150%"] = df["preco_compra"] * 2.5



df['preco_novo'] = np.where(
    df['compra+150%'] < df['pp+20%'], # se o preço de compra + 150% for menor que o preço da pp + 20%
    (df['pp+20%']).astype(int) -0.01, # então o novo preço é o preço da pp + 20% arredondado para baixo
    np.where(
        (df['preco_compra']*2.3/df['preco_compra'] * 100) < 300, # se a rentabilidade do preço de compra + 130% for menor que 300%
        (df['preco_compra']*2.3).astype(int) -0.01, # então o novo preço é o preço de compra + 130% arredondado para baixo
        df['pp+20%'] # senão o novo preço é o preço da pp+20%
        )
    )
df['rentabilidade_nova'] = round(df['preco_novo'] / df['preco_compra'] * 100, 2)

# excluir linha se o preco de compra for  menor que 0.1
df = df.drop(df[df['preco_compra'] <= 0.1].index)


# ajustar o preco_novo se a rentabilidade for maior que 500%
df['preco_novo_ajustado'] = np.where(
    df['rentabilidade_nova'] > 500,
    (df['preco_venda'] *0.8 + 1).astype(int) -0.01,
    df['preco_novo']
)

# rentabilidade nova ajustada
df['rentab_nova_ajustada'] = round(df['preco_novo_ajustado'] / df['preco_compra'] * 100, 2)

# verificar se existe preços muito diferentes entre preco_pp e preco_novo_ajustado
df['preco_novo_ajustado_2'] = np.where(
    df['preco_novo_ajustado'] > df['preco_pp'] * 1.8,
    (df['preco_pp'] * 1.25).astype(int) -0.01,
    df['preco_novo_ajustado']
)

# rentabilidade nova ajustada 2
df['rentab_nova_ajustada_2'] = round((df['preco_novo_ajustado_2'] / df['preco_compra']-1) * 100, 2)

# limpa o dataframe e renomeia as colunas
df.drop(['pp+20%', 'compra+150%', 'preco_novo', 'rentabilidade_nova', 'preco_novo_ajustado', 'rentab_nova_ajustada', ], axis=1, inplace=True)
df.rename({'preco_novo_ajustado_2': 'preco_novo', 'rentab_nova_ajustada_2': 'rentab_nova_ajustada'}, axis=1, inplace=True)

df.sort_values(by=['rentab_nova_ajustada'], ascending=False, inplace=True)
df.to_excel('genericos_ems.xls', index=False, encoding='utf-8')
df.head(6)
